In [1]:
from lxml import html
import requests
import pandas as pd
import numpy as np
import re

from IPython.core.display import display, HTML # расширяем блокнот
display(HTML("<style>.container { width:90% !important; }</style>"))

pd.set_option('display.max_columns', None) # выводим все колонки

from IPython.core.interactiveshell import InteractiveShell # выводим все результаты из ячейки
InteractiveShell.ast_node_interactivity = "all"

In [4]:
page = requests.get('https://www.premierleague.com/clubs')
tree = html.fromstring(page.content)

In [5]:
linkLocation = tree.cssselect('.indexItem')

teamLinks = []

for i in range(0,20):

    temp = linkLocation[i].attrib['href']

    temp = "http://www.premierleague.com/" + temp

    temp = temp.replace("overview", "squad")

    teamLinks.append(temp)

In [6]:
playerLink1 = []
playerLink2 = []

for i in range(len(teamLinks)):

    squadPage = requests.get(teamLinks[i])
    squadTree = html.fromstring(squadPage.content)

    playerLocation = squadTree.cssselect('.playerOverviewCard')

    for i in range(len(playerLocation)):

        playerLink1.append("http://www.premierleague.com/" + playerLocation[i].attrib['href'])

        playerLink2.append(playerLink1[i].replace("overview", "stats"))

In [11]:
Name = []
Team = []
Apps = []
HeightCM = []
WeightKG = []

for i in range(len(playerLink1)):

    playerPage1 = requests.get(playerLink1[i])
    playerTree1 = html.fromstring(playerPage1.content)
    playerPage2 = requests.get(playerLink2[i])
    playerTree2 = html.fromstring(playerPage2.content)

    tempName = str(playerTree1.cssselect('div.name')[0].text_content())

    try:
        tempTeam = str(playerTree1.cssselect('.table:nth-child(1) .long')[0].text_content())
    except IndexError:
        tempTeam = str("BLANK")

    try:
        tempApps = playerTree2.cssselect('.statappearances')[0].text_content()
        tempApps = int(re.search(r'\d+', tempApps).group())
    except IndexError:
        tempApps = float('NaN')

    try:
        tempHeight = playerTree1.cssselect('.pdcol3 li:nth-child(1) .info')[0].text_content()
        tempHeight = int(re.search(r'\d+', tempHeight).group())
    except IndexError:
        tempHeight = float('NaN')

    try:
        tempWeight = playerTree1.cssselect('.pdcol3 li+ li .info')[0].text_content()
        tempWeight = int(re.search(r'\d+', tempWeight).group())
    except IndexError:
        tempWeight = float('NaN')


    Name.append(tempName)
    Team.append(tempTeam)
    Apps.append(tempApps)
    HeightCM.append(tempHeight)
    WeightKG.append(tempWeight)

In [13]:
df = pd.DataFrame(
    {'Name':Name,
     'Team':Team,
     'Apps':Apps,
     'HeightCM':HeightCM,
     'WeightKG':WeightKG})

df.head()

,Name,Team,Apps,HeightCM,WeightKG
0,Bernd Leno,Arsenal,91,190.0,83.0
1,RÃºnar Alex RÃºnarsson,Arsenal,1,186.0,76.0
2,Karl Hein,Arsenal,0,193.0,NaN
3,Mat Ryan,Arsenal (Loan),122,184.0,82.0
4,Arthur Okonkwo,Arsenal,0,NaN,NaN


In [15]:
len(df)

628

In [18]:
df

,Name,Team,Apps,HeightCM,WeightKG
0,Bernd Leno,Arsenal,91,190.0,83.0
1,RÃºnar Alex RÃºnarsson,Arsenal,1,186.0,76.0
2,Karl Hein,Arsenal,0,193.0,NaN
3,Mat Ryan,Arsenal (Loan),122,184.0,82.0
4,Arthur Okonkwo,Arsenal,0,NaN,NaN
...,...,...,...,...,...
623,Adama TraorÃ©,Wolverhampton Wanderers,255,178.0,72.0
624,FÃ¡bio Silva,Wolverhampton Wanderers,0,185.0,75.0
625,Theo Corbeanu,Wolverhampton Wanderers U21,116,NaN,NaN
626,Willian JosÃ©,Wolverhampton Wanderers (Loan),2,189.0,93.0


In [14]:
df[df['Name']=="Loris Karius"]["HeightCM"]

Series([], Name: HeightCM, dtype: float64)

In [ ]:
df.to_csv("EPLData.csv")